In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


def load_data(file_path):
    """Loads the dataset from the specified file path."""
    return pd.read_csv(file_path)


def encode_columns(df, categorical_columns, label_columns):
    """Encodes categorical and label columns in the DataFrame."""
    # One-hot encoding categorical columns
    encoder = OneHotEncoder(sparse_output=False, drop=None)

    for column in categorical_columns:
        one_hot_encoded = encoder.fit_transform(df[[column]])
        one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out([column]))
        df = pd.concat([df, one_hot_df], axis=1)
        df = df.drop([column], axis=1)

    # Label encoding label columns
    label_encoders = {}

    for col in label_columns:
        label_encoders[col] = LabelEncoder()
        df[col] = label_encoders[col].fit_transform(df[col])

    return df


def remove_outliers_iqr(df, column_name, threshold=1.5):
    """Removes outliers from the specified column using the IQR method."""
    # Calculate quartiles for the specified column
    q25 = df[column_name].quantile(0.25)
    q75 = df[column_name].quantile(0.75)
    iqr = q75 - q25

    # Calculate the lower and upper bounds for outliers using the IQR method
    lower_bound = q25 - threshold * iqr
    upper_bound = q75 + threshold * iqr

    # Filter the DataFrame to remove rows with values outside the lower and upper bounds
    df_filtered = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

    # Print the number of outliers removed and the size of the filtered DataFrame
    outliers_count = len(df) - len(df_filtered)
    print(f"Number of outliers removed for '{column_name}' using IQR method: {outliers_count}")
    print(f"Size of filtered DataFrame for '{column_name}' using IQR method: {len(df_filtered)}")

    return df_filtered


def save_data(df, file_path):
    """Saves the DataFrame to a CSV file."""
    df.to_csv(file_path, index=True)


# Main execution
if __name__ == "__main__":
    # Load data
    file_path = r"C:\Users\cease\Documents\Datasets\End to End Projects\Customer lifetime Value Data.csv"
    df = load_data(file_path)

    # Define columns for encoding and outlier removal
    categorical_columns = ['State', 'Employment Status', 'Location', 'Marital Status', 'Policy Type', 'Sales Channel', 
                          'Vehicle Class']
    label_columns = ['Response', 'Coverage', 'Education', 'Gender', 'Policy', 'Renew Offer Type', 'Vehicle Size']
    outlier_cols = ['Customer Lifetime Value', 'Monthly Premium Auto', 'Total Claim Amount']

    # Encode columns
    df_encoded = encode_columns(df.copy(), categorical_columns, label_columns)

    # Remove outliers
    for col in outlier_cols:
        df_encoded = remove_outliers_iqr(df_encoded, col)

    # Save processed data
    save_data(df_encoded, 'Insurance Customer Lifetime Value.csv')

Number of outliers removed for 'Customer Lifetime Value' using IQR method: 817
Size of filtered DataFrame for 'Customer Lifetime Value' using IQR method: 8317
Number of outliers removed for 'Monthly Premium Auto' using IQR method: 301
Size of filtered DataFrame for 'Monthly Premium Auto' using IQR method: 8016
Number of outliers removed for 'Total Claim Amount' using IQR method: 227
Size of filtered DataFrame for 'Total Claim Amount' using IQR method: 7789


In [4]:
print(df_encoded.columns)

Index(['Response', 'Coverage', 'Education', 'Gender', 'Income',
       'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy', 'Renew Offer Type',
       'Total Claim Amount', 'Vehicle Size', 'Customer Lifetime Value',
       'State_Arizona', 'State_California', 'State_Nevada', 'State_Oregon',
       'State_Washington', 'Employment Status_Disabled',
       'Employment Status_Employed', 'Employment Status_Medical Leave',
       'Employment Status_Retired', 'Employment Status_Unemployed',
       'Location_Rural', 'Location_Suburban', 'Location_Urban',
       'Marital Status_Divorced', 'Marital Status_Married',
       'Marital Status_Single', 'Policy Type_Corporate Auto',
       'Policy Type_Personal Auto', 'Policy Type_Special Auto',
       'Sales Channel_Agent', 'Sales Channel_Branch',
       'Sales Channel_Call Center', 'Sales Channel_Web',
       'Vehicle Class_Four-Door Car', 'Vehic